In [13]:
import numpy as np
import _pickle as pickle
import glob
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from music21 import note, chord, instrument, converter
from keras.layers import LSTM, Dense, Dropout, Activation, Input
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline 

/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# Set to true for debugging
debug = True

In [ ]:
# Main function to call all subfunctions in the notebook.
def train_network():
    # get data and convert it to notes
    notes, n_vocab = get_notes(quick=False)
    # prepare data
    mapped_notes, NetworkInput, NetworkOutut = prepare_data(notes, n_vocab)
    # get model
    model = get_model(NetworkInput.shape[1:], n_vocab)
    # train
    train(model, NetworkInput, NetworkOutut)
#--------------------------------------------------------------------
# uncomment after runing all cells.
#train_network()

In [ ]:
def get_notes(quick=False):
    ''' Read input midi files and convert them to notes
        Also quick refers to using the saved notes.pkl to retrieve notes instead of reading midi files.
    '''
    input_folder = './music/Mozart/'
    output_folder = './data/notes/Mozart2/'
    notes = {}
    vocabs = set()
    octaves = set()
    pitch = { p : 0 for p in ['A', 'B', 'B-', 'C', 'C#', 'D', 'E', 'E-', 'F', 'F#', 'G', 'G#'] }
    piece = {'octave' : 0, 'pitch':pitch.copy()}
    
    for index,file in enumerate(glob.glob(input_folder+'*.mid')):
        
        if index % 10 == 0 : print(index,end='')
        print('.',end='')
        midi = converter.parse(file)
        notes_to_parse  = []
        parts = instrument.partitionByInstrument(midi)
        
        if parts :
            piano_notes = parts.parts[0].recurse()
        else :
            print('\n*caution no parts are found.')
            piano_notes = parts.flat_notes
        
        # prepare notes
        notes_to_parse = []
        for element in piano_notes:
            if isinstance(element, note.Note):
                new_note = piece.copy()
                assert element.name in pitch.keys()
                new_note['pitch'][element.name] = 1
                new_note['octave'] = element.octave
                notes_to_parse.append(new_note)
                
            elif isinstance(element, chord.Chord):
                new_chord = piece.copy()
                for elementx2 in element.pitches :
                    assert elementx2.name in pitch.keys()
                    new_chord['pitch'][elementx2] = 1
                    
                notes_to_parse.append('.'.join(str(e) for e in element.pitches))
            else : # other parts we won't use them
                pass
            octabes.add(element.octave)
        
        #write notes to a file
        fn = output_folder + file[0].split('/')[-1].split('.')[0] + '.pkl'
        with open(fn, 'wb') as f:
            pickle.dump(notes_to_parse, f)
        
        # add notes to the dictionary
        notes[index] = notes_to_parse
        # add new vocabs to the set
        for element in notes_to_parse :
            vocabs.add(element)
    
    # write the big dictionary to the disk also
    with open(output_folder+'all_notes_dict.pkl', 'wb') as f :
        pickle.dump(notes, f)
    # write vocabs to disk
    with open(output_folder+'vocabs.pkl','wb') as f :
        pickle.dump(vocabs, f)
    print('\nThere are {} pieces and {} vocabs'.format(len(notes), len(vocabs)))
    print('data loaded properly and saved to disk.')
    return notes, vocabs
#--------------------------------------------------------
if debug : notes, vocabs = get_notes(quick=False)

In [6]:
pitch = { p:0
               for p in ['A', 'B', 'B-', 'C', 'C#', 'D', 'E', 'E-', 'F', 'F#', 'G', 'G#']
            }

In [10]:
pitch.copy()

{'A': 0,
 'B': 0,
 'B-': 0,
 'C': 0,
 'C#': 0,
 'D': 0,
 'E': 0,
 'E-': 0,
 'F': 0,
 'F#': 0,
 'G': 0,
 'G#': 0}

In [ ]:
def prepare_data(notes, n_vocab):
    ''' create input sequences and output notes '''
    sequence_length = 100
    NetworkInput = []
    NetworkOutput = []
    # create a mapping to the notes
    mapper = LabelEncoder()
    mapped_notes = mapper.fit_transform(notes)

    for i in range(len(notes)-sequence_length):
        in_seq = mapped_notes[i : i+sequence_length]
        out_note = mapped_notes[i+sequence_length]
        NetworkInput.append(in_seq)
        NetworkOutput.append(out_note)
    
    n_patterns = len(NetworkOutput)
    
    NetworkInput = np.reshape(NetworkInput, (n_patterns, sequence_length, 1))
    NetworkInput = NetworkInput / float(n_vocab)
    
    NetworkOutput = np.reshape(NetworkOutput, (-1,1))
    hotencoder = OneHotEncoder(sparse=False)
    _ = hotencoder.fit(mapped_notes.reshape(-1,1))
    NetworkOutput = hotencoder.transform(NetworkOutput)
    
    # save the mapper and hotencoder to disk for prediction.
    #with open('./data/mapper.pkl','wb') as f:
    #    pickle.dump(mapper, f)
    #with open('./data/hotencoder.pkl','wb') as f:
    #    pickle.dump(hotencoder, f)
    
    print('Input shape = ',NetworkInput.shape, '\nOutput shape = ', NetworkOutput.shape)
    return mapped_notes, NetworkInput, NetworkOutput
#---------------------------------------
if debug : mapped_notes, NetworkInput, NetworkOutput = prepare_data(notes, n_vocab)

In [ ]:
#with open('./checkpoints/1/loss_stack.pkl','wb') as f:
#    pickle.dump(loss_stack, f)

In [ ]:
#plt.plot(range(len(loss_stack)),loss_stack)
#plt.show()

In [17]:
with open('./data/notes/Mozart/vocabs.pkl', 'rb') as f:
    new_vocabs = pickle.load(f)

In [22]:
[ i for i in new_vocabs if '.' in i]

['F3.A3',
 'C#6.D6',
 'E2.A2',
 'E-4.A3.C4',
 'C5.E-4',
 'D5.C5',
 'D4.F#3.B3',
 'D3.B2',
 'C#4.E4.A4',
 'F3.A3.D3',
 'F5.A5',
 'G4.C#5.A4',
 'D4.F4.B-4',
 'E5.C#5.A5',
 'E-3.C4.B-3',
 'F#5.D5.A4.A5',
 'D4.F4',
 'E5.C5',
 'C5.E-5.F5',
 'A4.F#4',
 'C4.A3.F#3',
 'B-4.D5.F4',
 'A4.E5',
 'G4.F#4',
 'E-5.G#5',
 'G#5.D6.F6.F5',
 'G4.B-3',
 'G4.C#4',
 'G3.C4.E3',
 'B-4.E-5.D5',
 'B-4.F#5.C#5',
 'F#4.F#3',
 'C4.F4.A4',
 'B-3.G4',
 'A4.C#4.E4',
 'E2.E3',
 'B2.B1',
 'B4.G#5',
 'G3.C4',
 'B3.F3',
 'E6.E5',
 'D4.G4.B-3',
 'E-5.C6',
 'B3.F4.G3',
 'F#4.D5.B4',
 'E-5.G5.E-6',
 'G#4.G#5',
 'E3.G3.A2',
 'D5.A4',
 'C4.C5',
 'C5.E4.G4',
 'F#4.C4.E4',
 'E-3.B-3',
 'E-4.C5',
 'A4.C5.E-5',
 'E-4.F#3',
 'C#5.C#4',
 'D5.E5',
 'B3.D4.F4',
 'C4.A4',
 'F3.B3.D3',
 'G4.E4.A3',
 'F#3.A3.C4',
 'G#4.D5',
 'D3.F#3.A3',
 'C#5.E4',
 'F3.G#3.B3',
 'E5.G5.A4',
 'B-4.C#4',
 'A3.B3',
 'B-3.C4',
 'F4.G4.B4',
 'E4.B3.G3',
 'G#4.E-4',
 'E5.B-5.G5',
 'F3.D3.B-3',
 'C4.E-3',
 'F4.F3',
 'F#3.C4.D4.A3',
 'B-4.D5',
 'D6.B-5',
 'E-

In [11]:
names = set()
for p in piano :
    if isinstance(p, note.Note):
        names.add(p.name)
    if isinstance(p, chord.Chord):
        for pp in p.pitches:
            names.add(pp.name)

NameError: name 'piano' is not defined

In [ ]:
octaves = set()
for index,file in enumerate(glob.glob('./music/Mozart/*.mid')):
        
    if index % 10 == 0 : print(index,end='')
    print('.',end='')
    midi = converter.parse(file)
    notes_to_parse  = []
    parts = instrument.partitionByInstrument(midi)

    if parts :
        piano_notes = parts.parts[0].recurse()
    else :
        print('\n*caution no parts are found.')
        piano_notes = parts.flat_notes

    # prepare notes
    notes_to_parse = []
    for element in piano_notes:
        octaves.add(element.octave)

In [ ]:
names

In [23]:
octaves = set()
for index,file in enumerate(glob.glob('./music/vivaldi/*.mid')):
        
    if index % 10 == 0 : print(index,end='')
    print('.',end='')
    midi = converter.parse(file)
    notes_to_parse  = []
    parts = instrument.partitionByInstrument(midi)

    if parts :
        piano_notes = parts.parts[0].recurse()
    else :
        print('\n*caution no parts are found.')
        piano_notes = parts.flat_notes

    # prepare notes
    notes_to_parse = []
    for element in piano_notes:
        octaves.add(element.octave)

0.

AttributeError: 'Part' object has no attribute 'octave'

In [41]:
violin = parts.parts[2]

In [43]:
violin.write('midi', 'music/generated/violinvivalditest.mid')

'music/generated/violinvivalditest.mid'

In [ ]:
element.octave